In [ ]:
import time
import string
import nltk
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import pyhdb

start_time = time.time()

connection = pyhdb.connect(host="IP", port=30015, user="USERNAME", password="PASS")
query = "SQL QUERY TO PULL DATA"
dtProductDetails = pd.read_sql_query(query, connection)
dtProductDetails.head()

df = dtProductDetails[["PRODUCT_CODE", "DESC_DTL", "CATEGORY"]]
df

# to remove data in HTML tags
#import re
#CLEANR = re.compile('<.*?>|<img [^>]*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')

from bs4 import BeautifulSoup
def cleanhtml(raw_html):
    cleantext = BeautifulSoup(raw_html, "lxml").text
    return cleantext

df["DESC_DTL"] = df["DESC_DTL"].astype(str).apply(cleanhtml)
df

stopwords = nltk.corpus.stopwords.words('english')
def clean(doc): # doc is a string of text
    doc = doc.replace("</br>", " ")
    doc = "".join([char if char not in string.punctuation else " " for char in doc])
    #doc = "".join([char for char in doc if char not in string.punctuation])
    doc = " ".join([token for token in doc.split() if token not in stopwords])
    return doc.lower()

df["DESC_DTL"] = df["DESC_DTL"].astype(str).apply(clean)
df

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def fnLemmatize(doc): # doc is a string of text
    word_list = nltk.word_tokenize(doc)
    doc = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    return doc

df["DESC_DTL"] = df["DESC_DTL"].astype(str).apply(fnLemmatize)

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(analyzer='word', stop_words = None, ngram_range = (1,3), dtype = np.float32)
score = tfidf.fit_transform(df['DESC_DTL'])

dfScore = pd.DataFrame(score.toarray(), columns=tfidf.get_feature_names())

result = pd.concat([df,pd.DataFrame(dfScore.apply(lambda x:list(dfScore.columns[np.array(x).argsort()[::-1][:19]]), axis=1).values.tolist(),  columns=['c1', 'c2', 'c3','c4', 'c5', 'c6','c7', 'c8', 'c9','c10', 'c11', 'c12','c13', 'c14', 'c15','c16', 'c17', 'c18','c19'])], axis=1)


#df.columns.values
result["1-5"] = df["CATEGORY"].astype(str) + "," + result[['c1', 'c2', 'c3','c4']].apply(",".join, axis=1)
result["6-10"] = result[['c5', 'c6', 'c7','c8', 'c9']].apply(",".join, axis=1)
result["11-15"] = result[['c10', 'c11', 'c12','c13', 'c14']].apply(",".join, axis=1)
result["16-20"] = result[['c15','c16', 'c17', 'c18','c19']].apply(",".join, axis=1)

#result.drop(['c1', 'c2', 'c3','c4', 'c5', 'c6','c7', 'c8', 'c9','c10', 'c11', 'c12','c13', 'c14', 'c15','c16', 'c17', 'c18','c19', 'c20'], axis=1)
#result

result = result.drop(result.iloc[:,2:22],axis = 1)
result

end_time = time.time()
elapsed_time = end_time - start_time
print('Execution time:', elapsed_time, 'seconds')